# TODO: Document
# TODO: Cleanup
# TODO: Merge with UserCF if possible

In [1]:
# CHANGE THIS PARAMETER
recommendee = "taapaye"

In [2]:
import functools
import os
import pickle
import random

import numpy as np
import pandas as pd
import scipy.stats as st
import seaborn as sns
import statsmodels.formula.api as smf
from matplotlib import pyplot as plt


@functools.wraps(smf.ols)
def lm(*args, **kwargs):
    return smf.ols(*args, **kwargs).fit()

In [3]:
np.random.seed(0)

In [4]:
os.chdir(f"../../data/recommendations/{recommendee}")

In [5]:
filtered_df = pickle.load(open("../../processed_data/user_anime_lists.pkl", "rb"))

In [6]:
user_df = pickle.load(open("user_anime_list.pkl", "rb"))
filtered_df = filtered_df.loc[lambda x: ~x["username"].isin(user_df.username)]
filtered_df = pd.concat([filtered_df, user_df], ignore_index=True)

In [24]:
filtered_df = filtered_df.set_index('username')

In [8]:
# average_rating = filtered_df["my_score"].mean()
# user_bias = (
#     pd.DataFrame(filtered_df.groupby("username")["my_score"].mean()).rename(
#         {"my_score": "user_bias"}, axis=1
#     )
#     - average_rating
# )
# anime_bias = (
#     pd.DataFrame(filtered_df.groupby("anime_id")["my_score"].mean()).rename(
#         {"my_score": "anime_bias"}, axis=1
#     )
#     - average_rating
# )

In [9]:
# filtered_df = filtered_df.merge(anime_bias, on=["anime_id"]).merge(
#     user_bias, on=["username"]
# )
# filtered_df["normalized_score"] = (
#     filtered_df["my_score"]
#     - filtered_df["anime_bias"]
#     - filtered_df["user_bias"]
#     - average_rating
# )
# filtered_df["orig_normalized_score"] = filtered_df["normalized_score"]
# filtered_df = filtered_df.set_index("username")
# filtered_df = filtered_df.dropna()

In [10]:
# filtered_df = filtered_df.rename({'score': 'my_score'}).drop('score_var', axis=1)

In [17]:
def prepare_prediction(recommendee, neighborhood):
    pred_df = pd.DataFrame()
    pred_df["delta"] = neighborhood.groupby("anime_id").apply(
        lambda x: np.dot(x["score"], x["corr"]) / x["corr"].abs().sum()
    )
    pred_df = pred_df.dropna()
    return pred_df

In [18]:
def get_squared_error(df, pred_df, recommendee):
    recommendee_df = pred_df.loc[
        pred_df.index.intersection(df.loc[recommendee].anime_id)
    ]
    recommendee_df = recommendee_df.merge(
        df.loc[recommendee].set_index("anime_id")["score"], on="anime_id"
    )
    errors = recommendee_df["pred_score"] - recommendee_df["score"]
    return np.dot(errors, errors)

In [13]:
all_corrs = pickle.load(open("../../processed_data/item_correlations.pkl", "rb"))
all_corrs["similarity"] = all_corrs["corr"].abs()
all_corrs = all_corrs.dropna()
all_corrs = all_corrs.loc[
    lambda x: x.index.get_level_values("anime_id_x")
    != x.index.get_level_values("anime_id_y")
]
all_corrs = all_corrs.sort_values(by="similarity")
all_corrs_nonneg = all_corrs.sort_values(by="corr")

In [19]:
def compute_accuracy_metrics(
    is_df, oos_df, corrs, recommendee, neighborhood_sizes, nonneg_corrs
):

    metrics = pd.DataFrame()
    for neighborhood_size in tqdm(reversed(sorted(neighborhood_sizes))):
        # extract model features
        corrs = corrs.groupby("anime_id_x").tail(neighborhood_size)

        score = is_df.loc[recommendee].merge(
                corrs.reset_index("anime_id_x"),
                left_on="anime_id",
                right_on="anime_id_y",
            )
        score = score.drop("anime_id", axis=1).rename(
                {"anime_id_x": "anime_id"}, axis=1
            )
        pred_df = prepare_prediction(recommendee, score)

        # train linear model
        recomendee_seen_shows = is_df.loc[recommendee].merge(pred_df, on=["anime_id"])
        model = lm("score ~ delta + 0", recomendee_seen_shows)

        # inference
        pred_df["pred_score"] = model.predict(pred_df)
        is_pred_df = pred_df.loc[
            lambda x: x.index.isin(is_df.loc[recommendee].anime_id)
        ]
        oos_pred_df = pred_df.loc[lambda x: x.index.isin(oos_df.anime_id)]

        # compute coverage
        is_coverage = len(is_pred_df) / len(is_df.loc[recommendee])
        oos_coverage = len(oos_pred_df) / len(oos_df)

        # compute rmse
        missing_is = is_df.loc[recommendee].loc[
            lambda x: ~x.anime_id.isin(is_pred_df.index)
            & ~x.anime_id.isin(oos_df.anime_id)
        ]
        missing_oos = oos_df.loc[lambda x: ~x.anime_id.isin(oos_pred_df.index)]
        is_se = get_squared_error(is_df, is_pred_df, recommendee)
        oos_se = get_squared_error(oos_df, oos_pred_df, recommendee)
        missing_is_se = np.dot(
            missing_is["score"], missing_is["score"]
        )
        missing_oos_se = np.dot(
            missing_oos["score"], missing_oos["score"]
        )
        is_rmse = np.sqrt((is_se + missing_is_se) / len(is_df.loc[recommendee]))
        oos_rmse = np.sqrt((oos_se + missing_oos_se) / len(oos_df))
        metrics = metrics.append(
            {
                "neighborhood_size": neighborhood_size,
                "nonneg_corrs": nonneg_corrs,
                "is_rmse": is_rmse,
                "is_coverage": is_coverage,
                "oos_rmse": oos_rmse,
                "oos_coverage": oos_coverage,
            },
            ignore_index=True,
        )

    return metrics

In [ ]:
%%time
from tqdm import tqdm

K = 10
base = np.sqrt(2)

errors_by_neighborhood_size = []
max_size = len(filtered_df.anime_id.unique())
neighborhood_sizes = [
    int(base ** i) for i in range(int(np.log(max_size) / np.log(base)) + 1)
] + [max_size]
neighborhood_sizes = sorted(list(set(neighborhood_sizes)))
splits = np.array_split(filtered_df.loc[recommendee].sample(frac=1), K)

for split in tqdm(splits):
    oos_df = split
    is_df = filtered_df.loc[
        lambda x: ~(
            (x.index.get_level_values("username") == recommendee)
            & x.anime_id.isin(oos_df.anime_id)
        )
    ]
    # we take copies as a safety precaution
    errors_by_neighborhood_size.append(
        compute_accuracy_metrics(
            is_df.copy(),
            oos_df.copy(),
            all_corrs.copy(),
            recommendee,
            neighborhood_sizes,
            False,
        )
    )
    errors_by_neighborhood_size.append(
        compute_accuracy_metrics(
            is_df.copy(),
            oos_df.copy(),
            all_corrs_nonneg.copy(),
            recommendee,
            neighborhood_sizes,
            True,
        )
    )    

,anime_id,score,score_var
username,,,
taapaye,5420,0.109400,2.056788
taapaye,36838,-0.353823,1.976644
taapaye,28623,-1.748317,1.924521
taapaye,4280,-1.622470,1.596876
taapaye,13601,-1.031269,1.731997


,anime_id,score,score_var
username,,,
taapaye,30503,0.059305,1.507792
taapaye,323,1.258549,2.271494
taapaye,528,-0.062043,2.005560
taapaye,5114,-0.772564,1.752228
taapaye,16001,0.273911,1.708916


,anime_id,score,score_var
username,,,
taapaye,35062,-1.700765,1.768768
taapaye,28171,-0.025795,1.628738
taapaye,28701,-1.952292,1.713641
taapaye,6594,-2.981564,1.909471
taapaye,5040,0.032081,1.712585


,anime_id,score,score_var
username,,,
taapaye,8888,-3.101006,1.639319
taapaye,34280,2.012862,1.834370
taapaye,33728,-0.730346,1.571130
taapaye,18115,-0.042087,1.555609
taapaye,36296,1.395476,1.695687


,anime_id,score,score_var
username,,,
taapaye,33255,0.013209,1.719517
taapaye,31181,0.538033,1.655000
taapaye,5152,0.289241,1.939608
taapaye,32105,-1.436286,2.092113
taapaye,4282,-2.275584,1.720773


...

,anime_id,score,score_var
username,,,
taapaye,1177,2.485375,2.454767
taapaye,1887,2.338101,2.381319
taapaye,31637,-0.472899,1.643465
taapaye,1535,-0.255907,1.947338
taapaye,918,0.434734,2.209360


,anime_id,score,score_var
username,,,
taapaye,31478,-1.131451,1.638361
taapaye,601,1.679031,2.555669
taapaye,4654,-2.247149,1.723218
taapaye,1915,0.639954,1.856582
taapaye,22199,-1.119843,2.052106


,anime_id,score,score_var
username,,,
taapaye,387,-1.638356,2.216275
taapaye,984,-0.463895,1.923812
taapaye,396,1.759479,1.821499
taapaye,440,-0.081028,2.550400
taapaye,20,-1.124995,2.207460


,anime_id,score,score_var
username,,,
taapaye,464,-0.344895,1.965831
taapaye,34280,2.012862,1.834370
taapaye,11741,0.762174,1.681724
taapaye,31490,-2.682841,1.693078
taapaye,32268,0.304184,1.640290


 10%|█         | 1/10 [17:33<2:38:02, 1053.63s/it]

In [ ]:
allerrors = pd.concat(errors_by_neighborhood_size, ignore_index=True)

In [ ]:
allerrors

In [ ]:
wide_data = pd.melt(allerrors, ["neighborhood_size", "nonneg_corrs"])

In [ ]:
wide_data.loc[lambda x: x.nonneg_corrs == True, "variable"] = (
    "nonneg_" + wide_data.loc[lambda x: x.nonneg_corrs == True, "variable"]
)

In [ ]:
plt.figure(figsize=(20, 10))
_ = sns.lineplot(
    x="neighborhood_size",
    y="value",
    hue="variable",
    data=wide_data.loc[lambda x: x.variable.str.contains("coverage")],
).set(xscale="log", title="Prediction Coverage")

In [ ]:
plt.figure(figsize=(20, 10))
_ = sns.lineplot(
    x="neighborhood_size",
    y="value",
    hue="variable",
    data=wide_data.loc[lambda x: x.variable.str.contains("rmse")],
).set(xscale="log", title="Root Mean Squared Error")

In [ ]:
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
allerrors.groupby(["nonneg_corrs", "neighborhood_size"]).mean().sort_values(
    by="oos_rmse"
).head(10)

In [ ]:
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
allerrors.groupby(["nonneg_corrs", "neighborhood_size"]).mean().xs(
    0, level="nonneg_corrs"
).sort_values(by="oos_rmse").head(10)